In [1]:
import numpy as np

def evaluate_solution(route_perm, distances, demands_ton, capacity=15):
    """
    Evalúa una permutación de ubicaciones (1..n) devolviendo la distancia total
    y la lista de sub-rutas (cada una inicia y termina en 0).
    - distances: matriz 58x58 de distancias (0 = depósito, 1-57 ubicaciones).
    - demands_ton: vector de demanda (57,) en toneladas.
    - capacity: capacidad del camión (15 t).
    """
    total_dist = 0.0
    routes = []
    current_demand = 0.0
    current_route = [0]   # iniciamos en el depósito
    current_node = 0
    
    for loc in route_perm:
        demand = demands_ton[loc - 1]  # índice 0->ubicación 1
        # Si cabe la demanda en la carga actual:
        if current_demand + demand <= capacity:
            total_dist += distances[current_node, loc]
            current_route.append(loc)
            current_demand += demand
            current_node = loc
        else:
            # Volver al depósito y cerrar ruta
            total_dist += distances[current_node, 0]
            current_route.append(0)
            routes.append(current_route)
            
            # Iniciar nueva ruta desde el depósito
            current_route = [0]
            current_demand = demand
            total_dist += distances[0, loc]
            current_route.append(loc)
            current_node = loc
    
    # Cerrar la última ruta volviendo al depósito
    total_dist += distances[current_node, 0]
    current_route.append(0)
    routes.append(current_route)
    return total_dist, routes


In [12]:
def decode_permutation(keys):
    """Convierte un array de claves (random-key) en una permutación 1..n."""
    idx = np.argsort(keys)
    # Sumamos 1 para mapear de 0-based a nodos 1..n
    perm = [i+1 for i in idx]  
    return perm


In [22]:
def evaluar_ruta(ruta, dist_matrix):
    # distancia desde depósito (índice 0) hasta primer cliente
    coste = dist_matrix[0, ruta[0]]  
    # distancia entre clientes consecutivos
    for i in range(len(ruta)-1):
        coste += dist_matrix[ruta[i], ruta[i+1]]
    # distancia desde último cliente de vuelta al depósito
    coste += dist_matrix[ruta[-1], 0]
    return coste


In [2]:
# jaya
import random

def jaya_vrp(distances, demands_ton, pop_size=30, max_iter=1000):
    """
    Implementación del algoritmo Jaya para VRP usando codificación random-key.
    Retorna la mejor ruta (lista de sub-rutas) y la distancia total.
    """
    n = demands_ton.shape[0]  # 57 ubicaciones
    # Inicializar población de claves aleatorias
    population = np.random.rand(pop_size, n)
    distances_pop = np.zeros(pop_size)
    # Evaluar población inicial
    for i in range(pop_size):
        perm = decode_permutation(population[i])
        distances_pop[i], _ = evaluate_solution(perm, distances, demands_ton)
    # Identificar mejor y peor
    best_idx = np.argmin(distances_pop)
    worst_idx = np.argmax(distances_pop)
    best_dist = distances_pop[best_idx]
    best_keys = population[best_idx].copy()
    
    # Iteraciones del algoritmo
    for it in range(max_iter):
        for i in range(pop_size):
            X = population[i]
            new_X = np.zeros(n)
            # Actualizar cada clave de la solución i
            for j in range(n):
                r1 = random.random()
                r2 = random.random()
                Xb = best_keys[j]
                Xw = population[worst_idx][j]
                new_X[j] = X[j] + r1*(Xb - abs(X[j])) - r2*(Xw - abs(X[j]))
            # Evaluar solución nueva
            perm_new = decode_permutation(new_X)
            new_dist, _ = evaluate_solution(perm_new, distances, demands_ton)
            if new_dist < distances_pop[i]:
                # Reemplazar si es mejor
                population[i] = new_X
                distances_pop[i] = new_dist
        # Actualizar mejor y peor de la población
        best_idx = np.argmin(distances_pop)
        worst_idx = np.argmax(distances_pop)
        if distances_pop[best_idx] < best_dist:
            best_dist = distances_pop[best_idx]
            best_keys = population[best_idx].copy()
    # Decodificar y devolver la mejor ruta encontrada
    best_perm = decode_permutation(best_keys)
    best_distance, best_routes = evaluate_solution(best_perm, distances, demands_ton)
    return best_routes, best_distance


In [3]:
# Lobo gris
def gwo_vrp(distances, demands_ton, n_wolves=20, max_iter=500):
    """
    Grey Wolf Optimizer adaptado al VRP con codificación random-key.
    Retorna la mejor ruta y distancia.
    """
    n = demands_ton.shape[0]
    # Población inicial de lobos (claves aleatorias)
    wolves = np.random.rand(n_wolves, n)
    fitness = np.zeros(n_wolves)
    for i in range(n_wolves):
        perm = decode_permutation(wolves[i])
        fitness[i], _ = evaluate_solution(perm, distances, demands_ton)
    # Ordenar lobos por fitness (menor distancia es mejor)
    sorted_idx = np.argsort(fitness)
    wolves, fitness = wolves[sorted_idx], fitness[sorted_idx]
    # Definir alpha, beta, delta iniciales
    alpha = wolves[0].copy()
    beta  = wolves[1].copy()
    delta = wolves[2].copy()
    
    for t in range(max_iter):
        # Factor de exploración a decrece de 2 a 0
        a = 2 - (2 * t / max_iter)
        for i in range(n_wolves):
            X = wolves[i]
            X1 = np.zeros(n); X2 = np.zeros(n); X3 = np.zeros(n)
            # Calcular nuevas posiciones influenciadas por α, β, δ
            for j in range(n):
                r1, r2 = random.random(), random.random()
                A1 = 2*a*r1 - a; C1 = 2*r2
                X1[j] = alpha[j] - A1 * abs(C1*alpha[j] - X[j])
                
                r1, r2 = random.random(), random.random()
                A2 = 2*a*r1 - a; C2 = 2*r2
                X2[j] = beta[j]  - A2 * abs(C2*beta[j]  - X[j])
                
                r1, r2 = random.random(), random.random()
                A3 = 2*a*r1 - a; C3 = 2*r2
                X3[j] = delta[j] - A3 * abs(C3*delta[j] - X[j])
            # Promedio de las tres influencias
            wolves[i] = (X1 + X2 + X3) / 3
        # Evaluar nueva población
        for i in range(n_wolves):
            perm = decode_permutation(wolves[i])
            fitness[i], _ = evaluate_solution(perm, distances, demands_ton)
        # Reordenar y actualizar α, β, δ
        sorted_idx = np.argsort(fitness)
        wolves, fitness = wolves[sorted_idx], fitness[sorted_idx]
        alpha, beta, delta = wolves[0].copy(), wolves[1].copy(), wolves[2].copy()
    # Decodificar la mejor solución (alpha)
    best_perm = decode_permutation(alpha)
    best_distance, best_routes = evaluate_solution(best_perm, distances, demands_ton)
    return best_routes, best_distance


In [4]:
#RS
import math

def simulated_annealing_vrp(distances, demands_ton, initial_temp=1000, cooling_rate=0.995, n_iter=10000):
    """
    Recocido Simulado para VRP. Retorna la mejor ruta y distancia encontrados.
    """
    n = demands_ton.shape[0]
    # Solución inicial: permutación aleatoria
    current_perm = list(range(1, n+1))
    random.shuffle(current_perm)
    current_dist, _ = evaluate_solution(current_perm, distances, demands_ton)
    best_perm = current_perm.copy()
    best_dist = current_dist
    T = initial_temp
    
    for it in range(n_iter):
        # Vecino: intercambiar dos posiciones aleatorias
        i, j = random.sample(range(n), 2)
        neighbor = current_perm.copy()
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        new_dist, _ = evaluate_solution(neighbor, distances, demands_ton)
        
        # Criterio de aceptación
        if new_dist < current_dist or random.random() < math.exp((current_dist - new_dist) / T):
            current_perm = neighbor
            current_dist = new_dist
            if new_dist < best_dist:
                best_perm = neighbor.copy()
                best_dist = new_dist
        # Enfriar temperatura
        T *= cooling_rate
        if T < 1e-6:
            break
    
    # Decodificar y devolver la mejor solución
    best_distance, best_routes = evaluate_solution(best_perm, distances, demands_ton)
    return best_routes, best_distance


In [14]:
import numpy as np
distances = np.load('matriz_distancias.npy' ) # incluye el nodo 0
distances

array([[0.        , 7.81255656, 7.9878503 , ..., 8.59298465, 8.42261352,
        9.49975125],
       [7.81255656, 0.        , 1.14852203, ..., 5.63736563, 4.24942668,
        4.4692347 ],
       [7.9878503 , 1.14852203, 0.        , ..., 6.77704361, 5.39714432,
        5.59696149],
       ...,
       [8.59298465, 5.63736563, 6.77704361, ..., 0.        , 1.46220377,
        2.10133471],
       [8.42261352, 4.24942668, 5.39714432, ..., 1.46220377, 0.        ,
        1.13299771],
       [9.49975125, 4.4692347 , 5.59696149, ..., 2.10133471, 1.13299771,
        0.        ]])

In [15]:
campanas_por_ubi = np.load('campanas_por_ubi.npy')
campanas_por_ubi

array([2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 2,
       3, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 2])

In [16]:
demands_ton  = 0.8*campanas_por_ubi
demands_ton 

array([1.6, 1.6, 1.6, 0.8, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6,
       1.6, 1.6, 1.6, 1.6, 3.2, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6,
       1.6, 1.6, 0.8, 0.8, 0.8, 1.6, 1.6, 1.6, 0.8, 1.6, 1.6, 0.8, 0.8,
       1.6, 1.6, 0.8, 1.6, 1.6, 2.4, 0.8, 0.8, 2.4, 2.4, 2.4, 2.4, 2.4,
       2.4, 2.4, 2.4, 0.8, 1.6])

In [19]:
from time import time
start = time()
rutas, distancia = jaya_vrp(distances, demands_ton, pop_size=30, max_iter=1000)
rutas = [[int(nodo) for nodo in ruta] for ruta in rutas]
print("Distancia total:", distancia)
for i, ruta in enumerate(rutas):
    print(f"Camión {i+1}: {ruta}")
print("Tiempo de ejecución:", time() - start, "segundos")


Distancia total: 315.31929293212727
Camión 1: [0, 5, 20, 14, 9, 45, 16, 10, 47, 2, 0]
Camión 2: [0, 6, 33, 34, 32, 55, 21, 41, 24, 0]
Camión 3: [0, 11, 15, 19, 17, 50, 56, 49, 57, 0]
Camión 4: [0, 3, 31, 25, 39, 4, 40, 29, 38, 52, 22, 7, 0]
Camión 5: [0, 37, 36, 13, 28, 8, 26, 23, 48, 0]
Camión 6: [0, 27, 51, 54, 43, 12, 53, 44, 30, 0]
Camión 7: [0, 42, 1, 46, 18, 35, 0]
Tiempo de ejecución: 9.893783807754517 segundos


In [20]:
rutas, distancia = gwo_vrp(distances, demands_ton, n_wolves=20, max_iter=500)
rutas = [[int(nodo) for nodo in ruta] for ruta in rutas]
print("Distancia total:", distancia)
for i, ruta in enumerate(rutas):
    print(f"Camión {i+1}: {ruta}")

Distancia total: 250.19152692591518
Camión 1: [0, 6, 7, 23, 24, 25, 37, 32, 35, 28, 0]
Camión 2: [0, 14, 16, 45, 50, 10, 20, 54, 29, 0]
Camión 3: [0, 33, 48, 15, 49, 22, 30, 42, 47, 27, 0]
Camión 4: [0, 5, 13, 8, 40, 36, 38, 34, 43, 55, 0]
Camión 5: [0, 4, 51, 26, 56, 57, 12, 21, 1, 3, 31, 0]
Camión 6: [0, 41, 39, 2, 17, 19, 53, 11, 9, 44, 0]
Camión 7: [0, 52, 46, 18, 0]


In [21]:
rutas, distancia = simulated_annealing_vrp(distances, demands_ton, initial_temp=1000, cooling_rate=0.995, n_iter=10000)
rutas = [[int(nodo) for nodo in ruta] for ruta in rutas]
print("Distancia total:", distancia)
for i, ruta in enumerate(rutas):
    print(f"Camión {i+1}: {ruta}")


Distancia total: 203.13505573626685
Camión 1: [0, 54, 27, 21, 57, 10, 9, 44, 32, 0]
Camión 2: [0, 13, 5, 38, 14, 22, 49, 12, 43, 34, 0]
Camión 3: [0, 56, 33, 45, 50, 17, 15, 47, 2, 42, 39, 0]
Camión 4: [0, 28, 18, 16, 46, 11, 20, 35, 29, 23, 4, 0]
Camión 5: [0, 7, 37, 55, 19, 53, 48, 36, 0]
Camión 6: [0, 8, 40, 41, 31, 1, 52, 51, 30, 3, 0]
Camión 7: [0, 25, 24, 26, 6, 0]
